# Previsão do Preço de Casas

## Introdução

Este conjunto de dados sintéticos foi projetado para tarefas de regressão em ciência de dados, especificamente para prever preços de imóveis. Ele simula um mercado imobiliário real com 10.000 observações e 13 características, incluindo variáveis numéricas, ordinais e categóricas, tornando-o ideal para praticar limpeza de dados, engenharia de características e modelagem de regressão avançada (por exemplo, regressão linear, florestas aleatórias, reforço de gradiente).

A variável-alvo (price_k_usd) foi gerada com base em relações realistas e ponderadas com as características independentes, garantindo uma estrutura de correlação forte, mas ruidosa, semelhante aos dados reais do mercado.

## Análise Exploratória de Dados (EDA)

In [1]:
# Importando bibliotecas necessárias
import pandas as pd # Manipulação do DataFrame
import numpy as np # Manipulação numéricas
import matplotlib.pyplot as plt # Visualização dos dados

# Importando o DataFrame
housing = pd.read_csv('house_prices.csv')

# Visualizando as 5 primeiras linhas
housing.head()

,house_id,square_footage,number_of_rooms,number_of_bathrooms,lot_size_sqft,distance_to_city_center_km,year_built,has_garage,quality_score,school_rating,building_material,heating_type,price_k_usd
0,1000,1660,4,2.5,13501,28.8,2015,Yes,8,5,Stone,Gas,1077.78
1,1001,4572,5,3.5,10010,24.0,1986,No,6,4,Brick,Gas,1165.15
2,1002,3892,2,2.5,7959,12.6,1973,Yes,5,4,Wood,Electric,1077.08
3,1003,1266,4,2.0,9269,29.5,2011,Yes,5,4,Brick,Gas,919.52
4,1004,4244,5,2.5,9692,11.0,1970,Yes,3,2,Brick,Gas,1073.88


In [14]:
# Olhando os valores únicos da coluna 'building material' e 'heating_type'
print(housing['building_material'].unique())
print(housing['heating_type'].unique())

['Stone' 'Brick' 'Wood']
['Gas' 'Electric' 'Oil']


Nas colunas categóricas observa-se que há três colunas distintas. Em *'has_garage'* temos apenas duas opções, tem ou não garagem. Porém nas coluna *'building_material'* e *'heating_type'* temos mais de uma opção, o que sugere tratamento diferente para cada coluna.</br> Para resolver isso pensei em criar colunnas dummy(0 ou 1) para cada categoria dos dados.

In [30]:
# Criando as colunas dummies

# Colunas 'Stone', 'Brick' e 'Wood' para 'building_material'
housing['Stone'] = housing['building_material'].replace({'Stone': 1, 'Brick': 0, 'Wood': 0}) 
housing['Brick'] = housing['building_material'].replace({'Stone': 0, 'Brick': 1, 'Wood': 0}) 
housing['Wood'] = housing['building_material'].replace({'Stone': 0, 'Brick': 0, 'Wood': 1}) 
# Colunas 'Gas', 'Eletric' e 'Oil' para 'heating_type'
housing['Gas'] = housing['heating_type'].replace({'Gas': 1, 'Eletric': 0, 'Oil': 0}) 
housing['Eletric'] = housing['heating_type'].replace({'Gas': 0, 'Eletric': 1, 'Oil': 0}) 
housing['Oil'] = housing['heating_type'].replace({'Gas': 0, 'Eletric': 0, 'Oil': 1})

# Excluindo as colunas 
housing = housing.drop(columns=['heating_type', 'building_material'])

# Verificando o novo dataset
housing.head()

KeyError: 'building_material'

In [2]:
# Informações técnicas do DataFrame
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   house_id                    10000 non-null  int64  
 1   square_footage              10000 non-null  int64  
 2   number_of_rooms             10000 non-null  int64  
 3   number_of_bathrooms         10000 non-null  float64
 4   lot_size_sqft               10000 non-null  int64  
 5   distance_to_city_center_km  10000 non-null  float64
 6   year_built                  10000 non-null  int64  
 7   has_garage                  10000 non-null  object 
 8   quality_score               10000 non-null  int64  
 9   school_rating               10000 non-null  int64  
 10  building_material           10000 non-null  object 
 11  heating_type                10000 non-null  object 
 12  price_k_usd                 10000 non-null  float64
dtypes: float64(3), int64(7), object(

Não há colunas com dados nulos, sendo 3 colunas categóricas e 10 numéricas.

In [3]:
# Informações estatísticas das colunas
housing.describe()

,house_id,square_footage,number_of_rooms,number_of_bathrooms,lot_size_sqft,distance_to_city_center_km,year_built,quality_score,school_rating,price_k_usd
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5999.50000,2906.349800,3.506800,2.277100,9523.834300,15.497810,1986.791200,6.508800,3.006800,1026.916494
std,2886.89568,1207.433822,1.114127,0.796322,3185.060018,8.362935,21.351946,2.287498,1.424485,192.863307
min,1000.00000,801.000000,2.000000,1.000000,4000.000000,1.000000,1950.000000,3.000000,1.000000,400.100000
25%,3499.75000,1847.500000,3.000000,2.000000,6774.750000,8.300000,1968.000000,4.000000,2.000000,889.025000
50%,5999.50000,2905.000000,4.000000,2.000000,9568.000000,15.400000,1987.000000,6.000000,3.000000,1027.705000
75%,8499.25000,3963.250000,5.000000,3.000000,12314.750000,22.700000,2005.000000,9.000000,4.000000,1167.295000
max,10999.00000,4999.000000,5.000000,4.000000,14999.000000,30.000000,2023.000000,10.000000,5.000000,1784.320000


## Modelo

In [10]:
# Importando bibliotecas necessárias
from sklearn.model_selection import train_test_split # Função para separar em treino e teste
from sklearn.linear_model import LinearRegression  # Importando o modelo de Regressão Linear
from sklearn.preprocessing import OneHotEncoder, StandardScaler # Funções para tratamento dos dados
from sklearn.compose import ColumnTransformer # Função para tratar as colunas
from sklearn.pipeline import Pipeline # Função para criar um pipeline dos dados

# Excluindo a coluna 'house_id'
housing = housing.drop(columns=['house_id'])

# Selecionando features e target
X = housing.drop(columns=['price_k_usd']) # features
y = housing['price_k_usd'] # target

# Separando colunas categóricas das numéricas
numerical_columns = [
categorical_features = X.select_dtypes(include='object').columns.tolist() # Colunas categóricas
numerical_features = X.select_dtypes(exclude='object').columns.tolist() # Colunas numéricas

# Instanciando as tranformações
preprocessing = ColumnTransformer(
    transformers=([('cat', OneHotEncoder(), categorical_features),
                   ('num', StandardScaler(), numerical_features)])
)

# Separando em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Printando as divisões
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

KeyError: "['house_id'] not found in axis"

In [ ]:
# Criando o pipeline dos dados
pipeline = Pipeline(steps=([
    ('processor', preprocessing),
    ('model', LinearRegression())]
    )
)

# Treinando o modelo
pipeline.fit(X_train, y_train)

# Prevendo
y_pred = pipeline.predict(X_test)

### Avaliação do modelo

In [6]:
# Importando as métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

# Instanciando as funções
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mrse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Printando os resultados
print(f'mae: {mae}')
print(f'mse: {mse}')
print(f'mrse: {mrse}')
print(f'mape: {mape}')
print(f'R-Quadrado: {r2}')

mae: 47.65560463222305
mse: 3867.2373370274304
mrse: 62.18711552265011
mape: 0.0479457608604484
R-Quadrado: 0.8965992271825525


Nosso modelo não performou do jeito que gostaria, por isso vou testar um modelo mais potente... O **XGBoost!**

## Treinando com o XGBoost

In [11]:
# Importanto a biblioteca
from xgboost import XGBRegressor

# Separando os tipos de colunas
categorical_xgb = X.select_dtypes(include='object').columns.tolist() # Colunas categóricas
numerical_xgb = X.select_dtypes(exclude='object').columns.tolist() # Colunas numéricas()
# Tratando as colunas separadamente
ohe = OneHotEncoder() 


# Já vamos começar a dividir em treino e teste
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y, test_size=0.2, random_state=123)

# Instanciando o modelo
model = XGBRegressor(objective='reg:squarederror', random_state=123)

# Treinando o modelo
model.fit(X_train_xgb, y_train_xgb)

# Realizando as predições
predictions = model.predict(X_test)